In [1]:
import pandas as pd
import os
from glob import glob
from sklearn.model_selection import train_test_split
import numpy as np
import tensorflow as tf
import tensorflow.keras as K
from tensorflow.keras import backend as Kb
import matplotlib.pyplot as plt

In [3]:
## Load the NIH data toa pandas dataframe
all_xray_df = pd.read_csv('../data/Data_Entry_2017.csv')
all_image_paths = {os.path.basename(x): x for x in glob(os.path.join('../../data','images*', '*', '*.png'))}

print('Total X-Ray images found:', len(all_image_paths))

Total X-Ray images found: 112120


In [4]:
all_xray_df['path'] = all_xray_df['Image Index'].map(all_image_paths.get)
pa_xray_df = all_xray_df.drop(all_xray_df.loc[all_xray_df['View Position']=='AP'].index)

In [5]:
label_set = pa_xray_df.rename(columns={'Finding Labels': 'labels'})
label_set = label_set.set_index('Image Index').labels.str.split('|', expand=True).stack().reset_index(level=1, drop=True).to_frame('labels')
label_set = pd.get_dummies(label_set, columns=['labels']).groupby(level=0).sum()
label_set.head()

,labels_Atelectasis,labels_Cardiomegaly,labels_Consolidation,labels_Edema,labels_Effusion,labels_Emphysema,labels_Fibrosis,labels_Hernia,labels_Infiltration,labels_Mass,labels_No Finding,labels_Nodule,labels_Pleural_Thickening,labels_Pneumonia,labels_Pneumothorax
Image Index,,,,,,,,,,,,,,,
00000001_000.png,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
00000001_001.png,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0
00000001_002.png,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
00000002_000.png,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
00000003_000.png,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


In [6]:
pa_xray_df.set_index('Image Index', inplace=True)
prepared_df = pa_xray_df.merge(label_set, left_index = True, right_index=True)
prepared_df.head()
#prepared_df.to_csv("prepared_df_VAE.csv")

,Finding Labels,Follow-up #,Patient ID,Patient Age,Patient Gender,View Position,OriginalImage[Width,Height],OriginalImagePixelSpacing[x,y],...,labels_Emphysema,labels_Fibrosis,labels_Hernia,labels_Infiltration,labels_Mass,labels_No Finding,labels_Nodule,labels_Pleural_Thickening,labels_Pneumonia,labels_Pneumothorax
Image Index,,,,,,,,,,,,,,,,,,,,,
00000001_000.png,Cardiomegaly,0,1,58,M,PA,2682,2749,0.143,0.143,...,0,0,0,0,0,0,0,0,0,0
00000001_001.png,Cardiomegaly|Emphysema,1,1,58,M,PA,2894,2729,0.143,0.143,...,1,0,0,0,0,0,0,0,0,0
00000001_002.png,Cardiomegaly|Effusion,2,1,58,M,PA,2500,2048,0.168,0.168,...,0,0,0,0,0,0,0,0,0,0
00000002_000.png,No Finding,0,2,81,M,PA,2500,2048,0.171,0.171,...,0,0,0,0,0,1,0,0,0,0
00000003_000.png,Hernia,0,3,81,F,PA,2582,2991,0.143,0.143,...,0,0,1,0,0,0,0,0,0,0


SyntaxError: invalid syntax (<ipython-input-6-66cfc81c6b76>, line 1)

In [9]:
prepared_df = pd.read_csv("../../data/formatted_image_database.csv", index_col="Image Index")

clean_data, unclean_data = train_test_split(prepared_df, test_size=0.01, random_state=42)

In [10]:
train_data_augmented = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True)

val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    featurewise_center=False,
    featurewise_std_normalization=False)

In [15]:
train_generator = train_data_augmented.flow_from_dataframe(
    clean_data, x_col='path', weight_col=None,
    target_size=(1024, 1024), color_mode='rgb',
    class_mode='raw', batch_size=16, shuffle=True, seed=42,
    save_to_dir=None, save_prefix='', save_format='png', subset=None,
    interpolation='nearest', validate_filenames=True)

validation_generator = val_datagen.flow_from_dataframe(
    unclean_data, x_col='path', weight_col=None,
    target_size=(1024, 1024), color_mode='rgb',
    class_mode='raw', batch_size=16, shuffle=True, seed=42,
    save_to_dir=None, save_prefix='', save_format='png', subset=None,
    interpolation='nearest', validate_filenames=True)

KeyError: 'class'

In [ ]:
batch = next(train_generator)

In [10]:
class Sampling(K.layers.Layer):
    def call(self, inputs):
        mean, log_sd = inputs
        return mean + tf.exp(0.5 * logvar) * tf.random.normal(mean = 0, stddev = 1, shape = mean.shape)

def create_encoder(in_shape, filters):
    inputs = K.layers.Input(shape = in_shape)
    for i in filters:
        x = K.layers.Conv2D(i,
                              kernel_initializer = 'he_normal',
                              strides = (1, 1),
                              padding = 'same',
                              kernel_size = (5, 5),
                              activation = 'elu')(x)
        x = K.layers.MaxPooling2D()(x)
        x = K.layers.BatchNormalization()(x)
    x = K.layers.Flatten()(x)
    x = K.layers.Dense(100, kernel_initializer = 'he_normal', activation = 'elu')(x)
    x = K.layers.BatchNormalization()(x)
    mean = K.layers.Dense(1, kernel_intializer = 'he_normal', activation = 'elu')(x)
    logvar = K.layers.Dense(1, kernel_intializer = 'he_normal', activation = 'elu')(x)
    sample = Sampling([mean, logvar])
    return K.Model(inputs = inputs, outputs = [mean, logvar, sample])
    
def create_decoder(filters):
    inputs = K.layers.Input(shape = (1))
    x = K.layers.Dense(100)(inputs)
    x = K.layers.Dense(32 * 32 * filters[0], kernel_initializer = 'he_normal', activation = 'elu')
    x = K.layers.Reshape((32, 32, filters[0]))
    for i in filters:
        x = K.layers.Conv2DTranspose(i,
                              kernel_initializer = 'he_normal',
                              strides = (1, 1),
                              padding = 'same',
                              kernel_size = (5, 5),
                              activation = 'elu')(x)
        x = K.layers.UpSampling2D()(x)
        x = K.layers.BatchNormalization()(x)
    x = K.layers.Conv2DTranspose(3, 
                                 kernel_size = 5, 
                                 kernel_initializer = 'he_normal', 
                                 strides = (1, 1), 
                                 activation = 'elu',
                                 padding = 'same')(x)
    return K.Model(inputs = inputs, outputs = x)

In [11]:
class BetaVAE(K.Model):
    def __init__(self, im_shape, filters, beta,**kwargs):
        super().__init__(**kwargs)
        self.encoder = create_encoder(im_shape, filters)
        self.decoder = create_decoder(filters)
        self.beta = beta
        self.kl_div_tracker = K.metrics.Mean(name = 'kl_loss')
        self.recon_loss_tracker = K.metrics.Mean(name = 'recon_loss')
        self.total_loss_tracker = K.metrics.Mean(name = 'total_loss')
        
    def call(self, inputs):
        with tf.GradientTape() as tape:
            mean, logvar, sample = self.encoder(inputs)
            recon = self.decoder(sample)
            recon_loss = tf.math.reduce_mean(
                tf.math.reduce_sum(
                    K.losses.binary_crossentropy(inputs, recon)
                )
            )
            kl_loss = 0.5 * (tf.exp(logvar) + tf.square(mean) - 1 - logvar)
            total_loss = recon_loss + self.beta * kl_loss
        gradients = tape.gradients(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_weights))
        self.kl_div_tracker.update_state(kl_loss)
        self.recon_loss_tracker.update_state(recon_loss)
        self.total_loss_tracker.update_state(total_loss)
        return {
            "loss" : self.total_loss_tracker.result(),
            "recon_loss" : self.recon_loss_tracker.result(),
            "kl_loss" : self.kl_div_tracker.result()}

In [ ]:
model = BetaVAE(im_shape = (1024, 1024, 3), filters = [8, 16, 32, 64, 100, 128], beta = 0.9)
model.compile(optimizer = 'adam')
model.fit(x = batch, epochs = 100)